<a href="https://colab.research.google.com/github/IndraMadika/ML_Study/blob/main/Cross_Validation/K_Fold_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, confusion_matrix

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical

In [2]:

from google.colab import files
uploaded = files.upload()


Saving cardio_train.csv to cardio_train.csv


In [7]:
rootPath='/content/'
dfDS = pd.read_csv(rootPath+"cardio_train.csv", delimiter=';')

In [8]:
#=====Split Dataset=======
X = dfDS.iloc[:, 1:len(dfDS.columns)-1].values
Y = dfDS["cardio"].values
xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size=0.20)

In [9]:
#=====Feature Scaling======
columnsToScale = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
scaler = MinMaxScaler(feature_range=(0,1))
xTrain = scaler.fit_transform(xTrain)
xTest = scaler.transform(xTest)

In [10]:
verbose, epochs, batch_size = 1, 100, 512
activationFunction='relu'

def getModel():
    model = Sequential()
    model.add(Dense(100, input_dim = xTrain.shape[1], activation=activationFunction))
    model.add(Dense(75, activation=activationFunction))
    model.add(Dense(50, activation=activationFunction))
    model.add(Dense(25, activation=activationFunction))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = getModel()

In [11]:
def showResults(test, pred):
    target_names = ['positive', 'negative']
    print(classification_report(test, pred, target_names=target_names))
    accuracy = accuracy_score(test, pred)
    precision=precision_score(test, pred, average='weighted')
    f1Score=f1_score(test, pred, average='weighted')
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("f1Score : {}".format(f1Score))
    cm=confusion_matrix(test, pred)
    print(cm)

In [12]:
#======== Cross Validation ===========
skf = StratifiedKFold(n_splits=5, shuffle=True)
skf.get_n_splits(xTrain, yTrain)
foldNum=0
for train_index, val_index in skf.split(xTrain, yTrain):
    foldNum+=1
    print("Results for fold",foldNum)
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]

    # one hot encode
    Y_train = to_categorical(Y_train)
    Y_val = to_categorical(Y_val)

    history = model.fit(X_train, Y_train,
                        validation_data = (X_val, Y_val),
                        epochs=epochs,
                        batch_size=batch_size)
    yPredict = model.predict(X_val)

    #Converting one hot encoded test label to label
    pred = np.argmax(yPredict, axis=1)
    val = np.argmax(Y_val, axis=1)

    showResults(val, pred)

Results for fold 1
Epoch 1/100
88/88 [==============================] - 3s 14ms/step - loss: 21.3043 - accuracy: 0.4989 - val_loss: 16.2166 - val_accuracy: 0.5029
Epoch 2/100
88/88 [==============================] - 1s 10ms/step - loss: 11.0249 - accuracy: 0.5099 - val_loss: 2.3248 - val_accuracy: 0.5027
Epoch 3/100
88/88 [==============================] - 0s 5ms/step - loss: 13.9691 - accuracy: 0.5039 - val_loss: 13.7813 - val_accuracy: 0.4985
Epoch 4/100
88/88 [==============================] - 0s 5ms/step - loss: 9.2044 - accuracy: 0.5101 - val_loss: 4.1411 - val_accuracy: 0.5054
Epoch 5/100
88/88 [==============================] - 0s 4ms/step - loss: 6.6629 - accuracy: 0.5077 - val_loss: 8.7434 - val_accuracy: 0.5029
Epoch 6/100
88/88 [==============================] - 0s 4ms/step - loss: 4.7995 - accuracy: 0.5136 - val_loss: 3.1457 - val_accuracy: 0.6063
Epoch 7/100
88/88 [==============================] - 0s 4ms/step - loss: 6.2915 - accuracy: 0.5224 - val_loss: 1.6425 - val_accu